# HPC as solutions for AI: TensorFlow

<p style='text-align: justify;'>
In this section, it will be shown how to optimize Tensorflow models, accelerating training and execution using GPUs.
</p>    

The principal gols are:

* **Understand** what is Tensorflow,
* **Learn** the basic concepts of Tensorflow for GPUs,
* **Create** a model using Tensorflow.

## What use Tensorflow in IA?

<p style='text-align: justify;'>
Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?
</p>    

## The solution: GPUs and TensorFlow

In [ ]:
##

##  ☆ Challenge: Zoo breakout!☆ 

Recently, there was an unexpected incident at the local zoo, **Orange Grove Zoo**: all the animals escaped from their enclosures and are now roaming freely in the zoo. To deal with this situation, we need your help to locate and classify the escaped animals, distinguishing each animal class and identifying possible vehicles that may be in the same environment.

You have been assigned as the person responsible for developing a computer vision system capable of identifying and classifying the escaped animals, as well as identifying the presence of vehicles in the images. For this challenge, we will use the CIFAR-10 dataset and the PyTorch library to train a deep learning model.

CIFAR-10 and CIFAR-100 datasets provide a comprehensive collection of 32x32 pixel images, grouped into 10 and 100 distinct classes, respectively.

- [CIFAR-10 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html): CIFAR-10 consists of 60,000 images, each belonging to one of the ten classes: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. This dataset offers a diverse set of images representing everyday objects.

- [CIFAR-100 Dataset](https://www.cs.toronto.edu/~kriz/cifar.html): CIFAR-100 expands upon the CIFAR-10 concept, containing 60,000 images as well. However, it introduces a more challenging task by categorizing images into 100 classes. These classes include various subcategories such as fruits, animals, vehicles, and more.

a) **Create** deep neural network model utilizing the PyTorch library for the classification of animals and vehicles on a CPU and on a GPU using CIFAR-10 dataset.

b) **Conduct** a comparative analysis between models trained on CPU and GPU to highlight disparities in results.

c) Now, use the CIFAR-100 dataset for the classification of animals and vehicles on a GPU. Would it be a good decision to use a GPU or a CPU for the training process?

### ☆ Solution ☆

#### ⊗ Importing Packages

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import time

#### ⊗ Verify the devices

It is very important, before trying to execute anything on any device, to verify if it is available and if TensorFlow can use it.

##### ⊗ Checking CPU

In [ ]:
# Checking if CPU is available
cpus = tf.config.experimental.list_physical_devices('CPU')
if not cpus:
    raise RuntimeError("No CPU available.")
else:
    print("CPU available")

##### ⊗ Checking GPU

In [ ]:
# Checking if GPU is available
gpus = tf.config.experimental.list_physical_devices('GPU')
if not gpus:
    raise RuntimeError("No GPU available.")
else:
    print("GPU available")

#### ⊗ Downloading the Dataset

Now we need to download the cifar10 dataset to be able to make predictions. Cifar10 is a dataset of labeled images, meaning that each image to be loaded already has a known label.

In [ ]:
# Loading the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

#### ⊗ Normalizing the Dataset

After downloading the entire set of images, we need to normalize them so that we can use them in our example.

In [ ]:
# Normalizing pixel values to the [0, 1] range
train_images, test_images = train_images / 255.0, test_images / 255.0

#### ⊗ Creating the Model

Now it is necessary to create the model for our neural network, notice that this step becomes extremely simple using the power of TensorFlow.

In [ ]:
# Creating the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

#### ⊗ Compiling

After creating the model, it needs to be compiled, just as we did in the first example with GPU.

In [ ]:
# Compiling the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#### ⊗ Training Function

Now it is important to define the training function of the model, which it will use to train using the Cifar10 dataset.

In [ ]:
# Function to train the model and measure time with progress
def train_model(device, train_images, train_labels):
    with tf.device(device):
        start_time = time.time()
        history = model.fit(train_images, train_labels, epochs=100, 
                    validation_data=(test_images, test_labels), verbose=1)
        end_time = time.time()
    
    return history, end_time - start_time

#### ⊗ Training the Model

<p style='text-align: justify;'> The next step is to perform the model training. Note that in the step below, we will use the GPU to train the model and then the CPU to train and compare their execution times. (Depending on the GPU and CPU of your machine, this step may take some time). </p>

In [ ]:
# Reduce the dataset for training and testing (e.g., 20% of the original data)
fraction_of_data = 1
num_samples = int(len(train_images) * fraction_of_data)

small_train_images = train_images[:num_samples]
small_train_labels = train_labels[:num_samples]
small_test_images = test_images[:num_samples]
small_test_labels = test_labels[:num_samples]

tf.keras.backend.clear_session()

# # Train the model on the CPU with the reduced dataset
# cpu_history, cpu_time = train_model('/CPU:0', small_train_images, small_train_labels)
# print(f"Training time on CPU: {cpu_time:.2f} seconds")

# Train the model on the GPU with the reduced dataset
gpu_history, gpu_time = train_model('/GPU:0', small_train_images, small_train_labels)
print(f"Training time on GPU: {gpu_time:.2f} seconds")

#### ⊗ Evaluating Model Accuracy

In the last step, we only need to evaluate the accuracy of the model we created. For didactic purposes, we will plot the result on a graph.

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("\nModel accuracy on the test set:", test_acc)

#### ⊗ Evaluating the Learning Curve

<p style='text-align: justify;'> After evaluating the accuracy that our model achieved, it is interesting to understand its learning curve during training, both on GPU and CPU. Below is the code responsible for plotting this curve on a graph. </p>

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(gpu_history.history['accuracy'], label='accuracy (GPU)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.plot(gpu_history.history['val_accuracy'], label='val_accuracy (GPU)')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

## Summary
In this notebook we have shown: 

- Install and use Tensorflow using **GPU** environments,
- Comparative performance tests between **CPU** and **GPU** on model training.

## Clear the memory
Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
#import IPython
#app = IPython.Application.instance()
#app.kernel.do_shutdown(True)

## Next

In this section, you learned how to use a Tensorflow in a simple example using a GPU environment. In the next section, you will learn about other applications that those devices can be pretty useful in [_03-hpc-simulations-pytorch.ipynb_](03-hpc-simulations-pytorch).